In [1]:
from skfeature.function.information_theoretical_based import MRMR

from skfeature.function.information_theoretical_based import MRMR
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mat4py import loadmat
import scipy.io
import os
import cv2
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def extract_data(dset):
    epoch1 = pd.read_excel(dset)
    labels_all = pd.read_excel('label_for_ut47_session0.xlsx')
    
#########################################################################################
#Cal of corr matrix
    epoch1_corrmat=epoch1.corr()
    cordf = pd.DataFrame(epoch1_corrmat)

#########################################################################################
#Cal of Mutual info score
    
    mi = epoch1.corr(method='spearman')
    mi_df = pd.DataFrame(mi)

    matrix = []

    for i in range(0, 64):
        row = []
        matrix.append(row)
        for j in range(0, 64):
            if i == j:
                row.append(1)
            elif i>j:
                row.append(cordf.iloc[i,j])
            else:
                row.append(mi_df.iloc[i,j])

    rel_df = pd.DataFrame(matrix)
    # cal of bandpowers

    import pyeeg

    def BandPower( x ):

        fs = 128
        band = [1,4,8,12,30]

        resp = pyeeg.bin_power(x,band,fs)

        return resp
    import numpy as np
    dfn=epoch1.to_numpy()
    dfn1 = dfn.T

    a=[]
    b=[]
    for i in range(len(dfn1)):
    
        fs = 200
        band = [1,4,8,13,30,100]

        resp = pyeeg.bin_power(dfn1[i],band,fs)
        a.append(resp[0])
        b.append(resp[1])
#pb = pd.DataFrame(a,columns=['Delta','Theta','Alpha','Beta','Gamma'])
    pr = pd.DataFrame(b,columns=['Ratio Delta','Ratio Theta','Ratio Alpha','Ratio Beta','Ratio Gamma'])
#prodata=pd.concat([pb,pr],axis=1)

###########################################################################################
#Cal of stats

    m =pd.DataFrame(epoch1.mean(),columns=['mean'])
    k =pd.DataFrame(epoch1.kurtosis(),columns=['kurtosis'])
    sk= pd.DataFrame(epoch1.skew(),columns=['skew'])
    st=pd.DataFrame(epoch1.std(),columns=['std'])
    all_stats = pd.concat([m,k,sk,st],axis=1)
#stats = pd.DataFrame([m,k,sk,st],columns=['mean','kurtosis','skew','std'])

###########################################################################################
#final test/train dataset

    final_df = pd.concat([rel_df,all_stats,pr],axis=1)
 # MRMR   
    
    #print(final_df.shape)
    
    return final_df

In [3]:
train_data = []

DATADIR = 'D:/subjFiles/UT047/behavioral/FR1/epochs eeg for session 0  uta47/data'
CATEGORIES=['Recalled','Not_recalled']

def create_traindata():
    
    for category in CATEGORIES:
        #print(category)
        path = os.path.join(DATADIR, category)
        #print(path)
        class_lab = CATEGORIES.index(category)
        
        
        for ex in os.listdir(path):
            try:
                
                ext_data = extract_data(ex)
                datarry = np.array(ext_data)
                
                
                
                
                train_data.append([datarry,class_lab])
                
            except Exception as e:
                pass
            

            
create_traindata()

In [4]:
import  random
#random.shuffle(train_data)
feat_all=[]
label_all=[]
for features,label in train_data:
    feat_all.append(features)
    label_all.append(label)
X = np.array(feat_all).reshape(300,4672)
y = np.array(label_all)
y_df = pd.DataFrame(label_all)
X.shape

(300, 4672)

In [5]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
new_df = scale.fit_transform(X)
#sns_plot=sns.heatmap(new_df,xticklabels=2,yticklabels=False,square=True)
#sns_plot.figure.savefig('testfig.png')



In [9]:
X = pd.DataFrame(new_df)
y = pd.DataFrame(y)
X.shape

(300, 4672)

In [10]:
X=X.values
X=X.astype(float)
y1= y.values
y2=y1[:,0]
n_samples, n_features = X.shape 
num_fea = 50
correct = 0
# obtain the index of each feature on the training set
idx,_,_ = MRMR.mrmr(X, y2, n_selected_features=num_fea)
 
    # obtain the dataset on the selected features
features = X[:, idx[0:num_fea]]

In [11]:
X1 = np.array(features)
X1.shape
#y1 = np.array(y)

(300, 50)

In [266]:
from sklearn.model_selection import train_test_split

#X_train,X_test,y_train,y_test=train_test_split(X1,y2,test_size=0.10,stratify = y2)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X1,y2,test_size=0.20,stratify=y2,random_state=0)

In [267]:
#Model2 Random forest with entropy
from sklearn.ensemble import RandomForestClassifier


model2=RandomForestClassifier(max_depth=10,n_estimators=2000,min_samples_split=3,criterion='gini')
model2.fit(X_train,y_train)
y_predrf=model2.predict(X_test)


#use print only if train_test_split is used
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predrf))
print(classification_report(y_test,y_predrf))
print(confusion_matrix(y_test,y_predrf))

0.5333333333333333
              precision    recall  f1-score   support

           0       0.22      0.09      0.12        23
           1       0.59      0.81      0.68        37

    accuracy                           0.53        60
   macro avg       0.41      0.45      0.40        60
weighted avg       0.45      0.53      0.47        60

[[ 2 21]
 [ 7 30]]


In [268]:
import xgboost as xgs
model3 = xgs.XGBClassifier(max_depth=5)
model3.fit(X_train,y_train)
ypred3 = model3.predict(X_test)

#use print only if train_test_split is used
#from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,ypred3))
print(classification_report(y_test,ypred3))
print(accuracy_score(y_test,ypred3))

[[ 7 16]
 [ 6 31]]
              precision    recall  f1-score   support

           0       0.54      0.30      0.39        23
           1       0.66      0.84      0.74        37

    accuracy                           0.63        60
   macro avg       0.60      0.57      0.56        60
weighted avg       0.61      0.63      0.60        60

0.6333333333333333


In [291]:
#model 4 SVM
from sklearn.svm import SVC
model4 = SVC(C=800,gamma=0.001)
model4.fit(X_train,y_train)
ypred4 = model4.predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report
#use print only if train_test_split is used
print(accuracy_score(y_test,ypred4))
print(confusion_matrix(y_test,ypred4))
print(classification_report(y_test,ypred4))

0.65
[[ 6 17]
 [ 4 33]]
              precision    recall  f1-score   support

           0       0.60      0.26      0.36        23
           1       0.66      0.89      0.76        37

    accuracy                           0.65        60
   macro avg       0.63      0.58      0.56        60
weighted avg       0.64      0.65      0.61        60



In [306]:
# finding best params for svm

from sklearn.model_selection import GridSearchCV
param_grid = {'C' : [0.1, 1, 10, 100,200,300,400,600,500,700,800,900, 1000 ], 'gamma': [1,0.1,0.001,0.0001,0.00001]}

In [307]:
grid = GridSearchCV(SVC(),param_grid,verbose=3,refit=True)
grid.fit(X_train,y_train)

C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 65 candidates, totalling 195 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.613, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.613, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.613, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.613, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.613, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.613, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ..........

[CV] .................... C=200, gamma=0.1, score=0.450, total=   0.0s
[CV] C=200, gamma=0.001 ..............................................
[CV] .................. C=200, gamma=0.001, score=0.650, total=   0.0s
[CV] C=200, gamma=0.001 ..............................................
[CV] .................. C=200, gamma=0.001, score=0.575, total=   0.0s
[CV] C=200, gamma=0.001 ..............................................
[CV] .................. C=200, gamma=0.001, score=0.625, total=   0.0s
[CV] C=200, gamma=0.0001 .............................................
[CV] ................. C=200, gamma=0.0001, score=0.613, total=   0.0s
[CV] C=200, gamma=0.0001 .............................................
[CV] ................. C=200, gamma=0.0001, score=0.613, total=   0.0s
[CV] C=200, gamma=0.0001 .............................................
[CV] ................. C=200, gamma=0.0001, score=0.613, total=   0.0s
[CV] C=200, gamma=1e-05 ..............................................
[CV] .

[CV] ................. C=500, gamma=0.0001, score=0.613, total=   0.0s
[CV] C=500, gamma=0.0001 .............................................
[CV] ................. C=500, gamma=0.0001, score=0.613, total=   0.0s
[CV] C=500, gamma=0.0001 .............................................
[CV] ................. C=500, gamma=0.0001, score=0.613, total=   0.0s
[CV] C=500, gamma=1e-05 ..............................................
[CV] .................. C=500, gamma=1e-05, score=0.613, total=   0.0s
[CV] C=500, gamma=1e-05 ..............................................
[CV] .................. C=500, gamma=1e-05, score=0.613, total=   0.0s
[CV] C=500, gamma=1e-05 ..............................................
[CV] .................. C=500, gamma=1e-05, score=0.613, total=   0.0s
[CV] C=700, gamma=1 ..................................................
[CV] ...................... C=700, gamma=1, score=0.562, total=   0.0s
[CV] C=700, gamma=1 ..................................................
[CV] .

[Parallel(n_jobs=1)]: Done 195 out of 195 | elapsed:    2.9s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 200, 300, 400, 600, 500, 700,
                               800, 900, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001, 1e-05]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [308]:
grid.best_params_

{'C': 800, 'gamma': 0.001}

In [309]:
gridpre = grid.predict(X_test)

In [310]:
print(classification_report(y_test,gridpre))
print(confusion_matrix(y_test,gridpre))

              precision    recall  f1-score   support

           0       0.60      0.26      0.36        23
           1       0.66      0.89      0.76        37

    accuracy                           0.65        60
   macro avg       0.63      0.58      0.56        60
weighted avg       0.64      0.65      0.61        60

[[ 6 17]
 [ 4 33]]


In [314]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model4, X1, y1, cv=20,scoring='accuracy')
score

C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

array([0.3125    , 0.625     , 0.75      , 0.5625    , 0.33333333,
       0.6       , 0.53333333, 0.53333333, 0.46666667, 0.6       ,
       0.6       , 0.6       , 0.6       , 0.53333333, 0.66666667,
       0.6       , 0.57142857, 0.57142857, 0.57142857, 0.64285714])

In [194]:
kf.get_n_splits(X)

5

In [198]:
for train_index, test_index in kf.split(X1):
    X_tr,X_te = X1[train_index],X1[test_index]
    y_tr, y_te = y1[train_index],y1[test_index]

In [200]:
X_te.shape

(60, 300)

In [202]:
X_tr.shape

(240, 300)

In [232]:
#model 4 SVM
from sklearn.svm import SVC
model4 = SVC(C=1000,gamma=0.0001)
model4.fit(X_tr,y_tr)
ypred4 = model4.predict(X_te)

from sklearn.metrics import confusion_matrix,classification_report
#use print only if train_test_split is used
print(accuracy_score(y_te,ypred4))
print(confusion_matrix(y_te,ypred4))
print(classification_report(y_te,ypred4))

0.7333333333333333
[[ 0  0]
 [16 44]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.73      0.85        60

    accuracy                           0.73        60
   macro avg       0.50      0.37      0.42        60
weighted avg       1.00      0.73      0.85        60



C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [229]:
from sklearn.ensemble import RandomForestClassifier


model2=RandomForestClassifier(max_depth=10,n_estimators=10000,min_samples_split=3,criterion='gini')
model2.fit(X_tr,y_tr)
y_predrf=model2.predict(X_te)


#use print only if train_test_split is used
from sklearn.metrics import accuracy_score
print(accuracy_score(y_te,y_predrf))
print(classification_report(y_te,y_predrf))
print(confusion_matrix(y_te,y_predrf))

C:\Users\AB Duke\Anaconda4\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.6333333333333333
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.63      0.78        60

    accuracy                           0.63        60
   macro avg       0.50      0.32      0.39        60
weighted avg       1.00      0.63      0.78        60

[[ 0  0]
 [22 38]]


C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [346]:
# example of bayesian optimization with scikit-optimize
from numpy import mean
from sklearn.datasets.samples_generator import make_blobs
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from skopt.space import Integer
from skopt.utils import use_named_args
from skopt import gp_minimize

search_space = [Integer(1, 100, name='C'), Integer(0.0001, 1, name='gamma')]
models = SVC()
@use_named_args(search_space)
def evaluate_model(**params):
    # something
    models.set_params(**params)
    # calculate 5-fold cross validation
    result = cross_val_score(models, X1, y1, cv=5, n_jobs=-1, scoring='accuracy')
    # calculate the mean of the scores
    estimate = mean(result)
    return 1.0 - estimate
# perform optimization
result = gp_minimize(evaluate_model, search_space)

# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: n_neighbors=%d, p=%d' % (result.x[0], result.x[1]))

ValueError: The gamma value of 0.0 is invalid. Use 'auto' to set gamma to a value of 1 / n_features.

In [341]:
search_space


[Integer(low=1, high=100), Integer(low=0.001, high=1)]

In [355]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

def sample_loss(params):
  C = params[0]
  gamma = params[1]

  # Sample C and gamma on the log-uniform scale
  modelD = SVC(C=10 ** C, gamma=10 ** gamma, random_state=12345)

  # Sample parameters on a log scale
  return cross_val_score(modelD,
                         X=X1,
                         y=y1,
                         scoring='accuracy',
                         cv=5)

In [ ]:
sample_loss([5,-5])

In [359]:
import sklearn.gaussian_process as gp

def bayesian_optimization(n_iters, sample_loss, xp, yp):


  # Define the GP
    kernel = gp.kernels.Matern()
    model = gp.GaussianProcessRegressor(kernel=kernel,
                                      alpha=1e-4,
                                      n_restarts_optimizer=10,
                                      normalize_y=True)
    for i in range(n_iters):
    # Update our belief of the loss function
        model.fit(xp, yp)

    # sample_next_hyperparameter is a method that computes the arg
    # max of the acquisition function
        next_sample = sample_next_hyperparameter(model, yp)

    # Evaluate the loss for the new hyperparameters
        next_loss = sample_loss(next_sample)

In [360]:
bayesian_optimization(5,[1,2],X1,y1)

NameError: name 'sample_next_hyperparameter' is not defined